In [267]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime

## 1ere hypothèse

- On supprime les colonnes NAN des heures
- On supprime la colonne date
- On remplace les NAN restants par des 0


In [268]:
path = "./data"
x_data = pd.read_csv(path + "/Xtrain_hgcGIrA.csv", sep=",")
x_data["station"] = x_data["station"].astype("category")
x_data = x_data.drop("date", axis=1)
y_data = pd.read_csv(
    path + "/Ytrain_yL5OjS4.csv", sep=",", usecols=[1]
)  # occupancy rate
prev_len = len(x_data)

In [269]:
x_data.dropna(subset=["hour"], inplace=True)
print(f"Retire {int((prev_len - len(x_data)) / prev_len*100)}% des données")

Retire 10% des données


In [270]:
x_data[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]] = x_data[
    ["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]
].fillna(0)
x_data

,train,way,station,hour,composition,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3
0,1,0,AD,06:00:00,2,0.000,0.00000,0.000000,0.201,0.138,0.091
1,1,0,AD,06:00:00,2,0.000,0.00000,0.000000,0.204,0.152,0.106
2,1,0,AD,06:00:00,2,0.000,0.00000,0.000000,0.213,0.153,0.111
3,1,0,AD,06:00:00,2,0.000,0.00000,0.000000,0.213,0.152,0.108
4,1,0,AD,06:00:00,2,0.000,0.00000,0.000000,0.210,0.147,0.096
...,...,...,...,...,...,...,...,...,...,...,...
31114,9,0,BE,08:00:00,2,0.152,0.18860,0.157000,0.080,0.100,0.000
31115,9,0,BE,08:00:00,2,0.153,0.18040,0.191000,0.089,0.121,0.000
31116,9,0,BE,08:00:00,2,0.166,0.14900,0.168000,0.099,0.129,0.000
31117,9,0,BE,08:00:00,2,0.182,0.19300,0.162000,0.074,0.101,0.000


In [271]:
# convert to int
x_data["hour"] = x_data["hour"].apply(lambda x: int(x[:2]))
# x_data["date"] = x_data["date"].apply(
#     lambda x: datetime.fromisoformat(x).timestamp()
# ) # convert to timestamp
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)

In [272]:
x_data["station"].value_counts()
x_data.dtypes

train            int64
way              int64
station           int8
hour             int64
composition      int64
p1q0           float64
p2q0           float64
p3q0           float64
p0q1           float64
p0q2           float64
p0q3           float64
dtype: object

In [273]:
y_data = y_data.iloc[x_data.index]

In [ ]:
x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(
    x_data, y_data
)

In [274]:
from sklearn.linear_model import LinearRegression

lin = LinearRegression()
lin.fit(x_data_train, y_data_train)

print("Score train", lin.score(x_data_train, y_data_train))
print("Score test", lin.score(x_data_test, y_data_test))

Score train 0.9102378482804379
Score test 0.9103045779515773


In [281]:
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean

my_kfold = KFold(n_splits=10, shuffle=True)
scores = cross_val_score(
    estimator=LinearRegression(), X=x_data, y=y_data, cv=my_kfold, n_jobs=-1
)  # permet de répartir les calculs sur plusieurs coeurs
print("Mean score, 10 Kfold", mean(scores))

TypeError: only integer scalar arrays can be converted to a scalar index